## Introduction


The Ant Colony Optimization (ACO) algorithm is a metaheuristic inspired by the foraging behavior of ants. In this implementation, ants construct solutions to a given optimization problem by iteratively moving through a solution space while depositing and sensing pheromone levels on edges. Over time, the algorithm converges towards optimal or near-optimal solutions, leveraging both local and global information.

## 1. Initialization:


In [ ]:
import random

In [ ]:
# Define your distance matrix
distance_matrix = [
    [0, 5, 15, 4],
    [5, 0, 4, 8],
    [15, 4, 0, 1],
    [4, 8, 1, 0]
]

In [ ]:
# Define model parameters
rho = 0.05  # Evaporation rate           ->	ρ
Q = 50      # Pheromone deposit constant -> Q

alpha = 0.1  # Pheromone influence       -> α
beta = 0.2   # Heuristic influence       -> β

initial_pheromone = 0.1  # Initial pheromone level
num_iterations = 10      # Number of iterations
num_ants = 6             # Number of ants

In [ ]:
def initialize_pheromones(num_nodes, initial_pheromone):
    """
    Initializes pheromone levels on each edge of the graph.
    """
    return [[initial_pheromone] * num_nodes for _ in range(num_nodes)]

## 2. Ant Movement:


```
       α       β
      τ   x   η
P = -------------
       α       β
    ∑ τ   x   η
```




In [ ]:
def calculate_probabilities(current_node, visited, distance_matrix, pheromone_matrix, alpha, beta):
    """
    Calculate probabilities for choosing the next node based on pheromone levels and distance information.
    """
    num_nodes = len(distance_matrix)
    probabilities = []
    denominator = 0

    # Calculate probabilities for choosing the next node
    for next_node in range(num_nodes):
        if not visited[next_node]:
            # Calculate pheromone and distance values
            pheromone = pheromone_matrix[current_node][next_node]     # Pheromone -> τ
            distance = distance_matrix[current_node][next_node]       # Distance  -> η
            # Calculate numerator
            numerator = pheromone ** alpha * distance ** beta
            # Accumulate the denominator
            denominator += numerator

            # Calculate probability for this next node
            probability = numerator / denominator

            # Store the probability and next node index
            probabilities.append((next_node, probability))
    return probabilities

In [ ]:
def generate_ant_solution(distance_matrix, pheromone_matrix, alpha, beta):
    """
    Generates a solution for a single ant by constructing a path based on pheromone levels and heuristic information.
    """
    num_nodes = len(distance_matrix)  # Number of nodes
    visited = [False] * num_nodes     # Track visited nodes
    ant_path = []                     # Ant's path

    current_node = random.randint(0, num_nodes - 1)  # Choose a random starting node
    visited[current_node] = True      # Mark starting node as visited
    ant_path.append(current_node)     # Add starting node to path

    # Continue until all nodes are visited
    while len(ant_path) < num_nodes:
        probabilities = calculate_probabilities(current_node, visited, distance_matrix, pheromone_matrix, alpha, beta)  # Calculate probabilities
        selected_node = random.choices([node for node, _ in probabilities], [prob for _, prob in probabilities])[0]     # Choose next node

        ant_path.append(selected_node)  # Move to selected node
        visited[selected_node] = True   # Mark selected node as visited
        current_node = selected_node    # Update current node

    return ant_path  # Return ant's path

## 3. Pheromone Update:


```                                                     
τ = (1-ρ) τ       &     	τ =  τ   + ∑ ∆τ   &      ∆𝜏 = Q / len
 new       old             new  old

```



In [ ]:
def update_pheromones(pheromone_matrix, ant_solutions, rho):
    """
    Updates pheromone levels on each edge of the graph.
    """
    num_nodes = len(pheromone_matrix)  # Number of nodes

    # Evaporate pheromones
    for i in range(num_nodes):
        for j in range(num_nodes):
          pheromone_matrix[i][j] = (1 - rho) * pheromone_matrix[i][j]

    # Deposit pheromones
    for ant_path in ant_solutions:
        for i in range(len(ant_path) - 1):
            # Calculate the change in pheromone level
            delta_tau = Q / len(ant_path)
            # Deposit pheromone
            pheromone_matrix[ant_path[i]][ant_path[i+1]] += delta_tau

## 4. Solution Evaluation:

In [ ]:
def calculate_path_distance(path, distance_matrix):
    distance = 0
    for i in range(len(path) - 1):
        # Add distance between current node and next node to total distance
        distance += distance_matrix[path[i]][path[i+1]]
    return distance

def choose_best_solution(ant_solutions, distance_matrix):
    # Find the solution with the minimum total distance
    best_solution = min(ant_solutions, key=lambda x: calculate_path_distance(x, distance_matrix))
    return best_solution

In [ ]:
def calculate_total_pheromone(best_solution, pheromone_matrix):
    """
    Calculates the total pheromone level along the edges of the best solution path.
    """
    total_pheromone = 0
    for i in range(len(best_solution) - 1):
        total_pheromone += pheromone_matrix[best_solution[i]][best_solution[i + 1]]
    return total_pheromone

In [ ]:
def print_shortest_path(iteration, shortest_path, distance_matrix, pheromone_matrix):
    """
    Print information about the shortest path found in an iteration.
    """
    # Calculate the distance
    distance = calculate_path_distance(shortest_path, distance_matrix)
    # Calculate the pheromone
    total_pheromone = int(calculate_total_pheromone(shortest_path, pheromone_matrix))

    path_str = " -> ".join(str(node) for node in shortest_path)
    print(f"Iteration {iteration + 1}: \tShortest Path: {path_str} \tDistance = {distance} \t\tPheromone Level: {total_pheromone}")


## 5. Main Algorithm Execution:

In [ ]:
def main():
    # Initialize pheromone matrix
    num_nodes = len(distance_matrix)
    pheromone_matrix = initialize_pheromones(num_nodes, initial_pheromone)

    # Main loop
    for iteration in range(num_iterations):
        # Generate solutions for each ant
        ant_solutions = [generate_ant_solution(distance_matrix, pheromone_matrix, alpha, beta) for _ in range(num_ants)]

        # Update pheromone levels based on the solutions
        update_pheromones(pheromone_matrix, ant_solutions, rho)

        # Choose the best solution found so far
        best_solution = choose_best_solution(ant_solutions, distance_matrix)

        # Print the shortest path found in this iteration
        print_shortest_path(iteration, best_solution, distance_matrix, pheromone_matrix)

    # Calculate distance of best solution
    best_distance = calculate_path_distance(best_solution, distance_matrix)

    # Calculate the total pheromone level along the edges of the best solution path
    total_pheromone_best_solution = int(calculate_total_pheromone(best_solution, pheromone_matrix))

    # Print the final best solution and its distance
    print("=" * 110)
    print(f"Final Solution: {best_solution}, \tDistance = {best_distance}, \tPheromone Level: {total_pheromone_best_solution}")

In [ ]:
if __name__ == "__main__":
    main()

Iteration 1: 	Shortest Path: 0 -> 3 -> 1 -> 2 	Distance = 16 		Pheromone Level: 112
Iteration 2: 	Shortest Path: 1 -> 2 -> 3 -> 0 	Distance = 9 		Pheromone Level: 159
Iteration 3: 	Shortest Path: 0 -> 3 -> 2 -> 1 	Distance = 9 		Pheromone Level: 133
Iteration 4: 	Shortest Path: 1 -> 0 -> 3 -> 2 	Distance = 10 		Pheromone Level: 221
Iteration 5: 	Shortest Path: 0 -> 3 -> 2 -> 1 	Distance = 9 		Pheromone Level: 255
Iteration 6: 	Shortest Path: 1 -> 0 -> 3 -> 2 	Distance = 10 		Pheromone Level: 408
Iteration 7: 	Shortest Path: 0 -> 1 -> 2 -> 3 	Distance = 10 		Pheromone Level: 286
Iteration 8: 	Shortest Path: 0 -> 1 -> 2 -> 3 	Distance = 10 		Pheromone Level: 359
Iteration 9: 	Shortest Path: 0 -> 1 -> 2 -> 3 	Distance = 10 		Pheromone Level: 441
Iteration 10: 	Shortest Path: 1 -> 2 -> 3 -> 0 	Distance = 9 		Pheromone Level: 414
Final Solution: [1, 2, 3, 0], 	Distance = 9, 	Pheromone Level: 414


## Conclusion:



*   After running the ACO algorithm, the final solution obtained is [1, 2, 3, 0], with a distance of 9 units.
*   This solution exhibits a significant level of pheromone deposition, reaching a total pheromone level of 414.
*   Through the collaborative effort of multiple ants and the pheromone updating mechanism, the algorithm effectively identifies a path that minimizes distance while maximizing pheromone presence, showcasing the efficacy of the ACO paradigm in solving complex optimization problems.



  